<a href="https://colab.research.google.com/github/rtrochepy/astronomer/blob/main/day4_by_rdtp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

API: Get set up

This section requires an API key with billing enabled. Start by installing and importing the Gemini API Python SDK.


In [1]:
%pip install -q -U 'google-generativeai>=0.8.3'

You do not need to restart the kernel.

In [2]:
import google.generativeai as genai
from IPython.display import Markdown, HTML, display


Set up your API key

To run the following cell, your API key must be stored it in a Kaggle secret named GOOGLE_API_KEY.

If you don't already have an API key, you can grab one from AI Studio. You can find detailed instructions in the docs.

To make the key available through Kaggle secrets, choose Secrets from the Add-ons menu and follow the instructions to add your key or enable it for this notebook.


In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)


Explore available models

Search grounding is a tool available in the -002 series of models. Find a model that supports it through the models.list endpoint. You can also find more information about different model capabilities on the models page.


In [4]:
for model in genai.list_models():
    if "002" in model.name:
        print(model.name)


models/gemini-1.5-pro-002
models/gemini-1.5-flash-002


Use search grounding

To enable search grounding, you specify it as a tool: google_search_retrieval. Like other tools, this can be supplied as a parameter to the model (to use on all chat turns or calls to generate_content), or it can be supplied per-turn to chat.send_message.


In [5]:
# Ask for information without search grounding.
model = genai.GenerativeModel("gemini-1.5-flash-002")

response = model.generate_content("When and where is Taylor Swift's next concert?")

Markdown(response.text)


I do not have access to real-time information, including live ticket sales and constantly updating concert schedules.  To find the date and location of Taylor Swift's next concert, you should check these resources:

* **Taylor Swift's Official Website:** This is the best place to find official tour dates.
* **Ticketmaster:**  A major ticket seller; search for Taylor Swift.
* **Live Nation:** Another large ticket vendor.
* **Other Ticket Resellers:**  Be cautious and only use reputable sites.


Remember that concert dates are subject to change, so always double-check close to the anticipated date.


Now try with grounding enabled.

In [6]:
# And now re-run the same query with search grounding enabled.
model = genai.GenerativeModel("gemini-1.5-flash-002", tools="google_search_retrieval")

response = model.generate_content("When and where is Taylor Swift's next concert?")
rc = response.candidates[0]

Markdown(rc.content.parts[0].text)


TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-002:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).

If you receive a 429 Resource has been exhausted error, you are likely using a free tier API key. You can choose to enable billing (but this will incur charges), or you can try the queries in Google AI Studio by following the Open in AI Studio links above.

Response metadata

When search grounding is used, the model returns extra metadata that includes links to search suggestions, supporting documents and information on how the supporting documents were used.

Each "grounding chunk" represents information retrieved from Google Search that was used in the grounded generation request. Following the URI will take you to the source.


In [7]:
chunks = rc.grounding_metadata.grounding_chunks
for chunk in chunks:
    print(chunk)


NameError: name 'rc' is not defined

As part of the response, there is a standalone styled HTML content block that you use to link back to relevant search suggestions related to the generation.

In [15]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

NameError: name 'rc' is not defined

The grounding_supports in the metadata provide a way for you to correlate the grounding chunks used to the generated output text.

In [ ]:
supports = rc.grounding_metadata.grounding_supports
for support in supports:
    print(support)


These supports can be used to highlight text in the response, or build tables of footnotes.

In [19]:
import io

markdown_buffer = io.StringIO()

# Print the text with footnote markers.
markdown_buffer.write("Supported text:\n\n")
for support in supports:
    markdown_buffer.write(" * ")
    markdown_buffer.write(
        response.text[support.segment.start_index : support.segment.end_index]
    )

    for i in support.grounding_chunk_indices:
        chunk = chunks[i].web
        markdown_buffer.write(f"<sup>[{i+1}]</sup>")

    markdown_buffer.write("\n\n")


# And print the footnotes.
markdown_buffer.write("Citations:\n\n")
for i, chunk in enumerate(chunks, start=1):
    markdown_buffer.write(f"* {i}: [{chunk.web.title}]({chunk.web.uri})\n")


Markdown(markdown_buffer.getvalue())


NameError: name 'supports' is not defined

Dynamic grounding

In a context where you may not know in advance whether to enable search grounding or not, you can provide the model with a threshold over which it will use search grounding. This is helpful in conversational contexts, where not every turn of conversation requires search data to support a response.

If you know whether to enable Search for any given chat turn, you can provide the tool explicitly.


In [20]:
nosearch_model = genai.GenerativeModel("gemini-1.5-flash-002")
chat = nosearch_model.start_chat()

# No search grounding.
r = chat.send_message("Hello friendly chatbot!")

# Enable search for just this turn.
r = chat.send_message(
    "Who took home the 2023 cricket world cup?", tools="google_search_retrieval"
)

Markdown(r.text)


TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-002:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).

In [ ]:
HTML(r.candidates[0].grounding_metadata.search_entry_point.rendered_content)

Or you can let the Gemini API calculate a likelihood that the response needs search backing, and define the threshold to use.

In [ ]:
# The dynamic retrieval score is a probability, so the threshold
# must also be bound by [0, 1].
search_config = {
    "dynamic_retrieval_config": {"mode": "MODE_DYNAMIC", "dynamic_threshold": 0.5}
}

maybe_search_model = genai.GenerativeModel(
    "gemini-1.5-flash-002", tools={"google_search_retrieval": search_config}
)

chat = maybe_search_model.start_chat()

r = chat.send_message("Hello friendly chatbot!")
rc = r.candidates[0]
score = rc.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f"First turn: {score=}")

r = chat.send_message("Who took home the 2023 cricket world cup?")
rc = r.candidates[0]
score = rc.grounding_metadata.retrieval_metadata.google_search_dynamic_retrieval_score
print(f"Second turn: {score=}")
print()

display(Markdown(r.text))


In [ ]:
HTML(rc.grounding_metadata.search_entry_point.rendered_content)

Further reading

When using search grounding, there are some specific requirements that you must follow, including when and how to show search suggestions, and how to use the grounding links. Be sure to read and follow the details in the search grounding capability guide and the search suggestions guide.